<a href="https://colab.research.google.com/github/UltraBirdTech/ml-security/blob/master/ch02/ml_security_cha02_05_spam_mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://githb.com/oreilly-japan/ml-security-jp/rw/master/ch02/enron1.zip
!unzip -q enron1.zip
!pip install optuna

--2022-02-26 04:17:56--  https://githb.com/oreilly-japan/ml-security-jp/rw/master/ch02/enron1.zip
Resolving githb.com (githb.com)... 162.210.199.65
Connecting to githb.com (githb.com)|162.210.199.65|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ww1.githb.com [following]
--2022-02-26 04:17:57--  http://ww1.githb.com/
Resolving ww1.githb.com (ww1.githb.com)... 199.59.240.200
Connecting to ww1.githb.com (ww1.githb.com)|199.59.240.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘enron1.zip.1’

enron1.zip.1            [ <=>                ]   1.23K  --.-KB/s    in 0s      

2022-02-26 04:17:57 (119 MB/s) - ‘enron1.zip.1’ saved [1261]

[enron1.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold, cross_validate
import os
import codecs

def init_lists(folder):
    key_list = []
    file_list = os.listdir(folder)
    for filename in file_list:
        f = codecs.open(folder + filename, 'r', encoding='utf8', errors='ignore')
        key_list.append(f.read())
    f.close()
    return key_list

all_mails = list()
spam = init_lists('./enron1/spam/')
ham = init_lists('./enron1/ham/')

all_mails = [(mail, '1') for mail in spam]
all_mails + [(mail, '0') for mail in ham]

import pandas as pd
df = pd.DataFrame(all_mails, columns=['text', 'label'])

from sklean.feature_extraction.text import TfidVectorizer

